In [ ]:
import gradio as gr
import os
from dotenv import load_dotenv
import requests
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
GITHUB_TOKEN=os.getenv("GITHUB_TOKEN")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
def parse_github_url(url):
    parts = url.rstrip("/").split("/")
    owner = parts[-2]
    repo = parts[-1]
    return owner, repo

In [ ]:
def get_repo_tree(owner, repo):
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    
    # Get default branch
    repo_data = requests.get(
        f"https://api.github.com/repos/{owner}/{repo}",
        headers=headers
    ).json()
    
    default_branch = repo_data["default_branch"]
    
    # Get tree recursively
    tree = requests.get(
        f"https://api.github.com/repos/{owner}/{repo}/git/trees/{default_branch}?recursive=1",
        headers=headers
    ).json()
    
    return tree.get("tree", [])

In [ ]:
IMPORTANT_EXTENSIONS = (
    ".py", ".js", ".ts", ".tsx", ".jsx",
    ".go", ".rs", ".java", ".cpp",
    ".json", ".toml", ".yaml", ".yml"
)

def filter_files(tree):
    return [
        file["path"]
        for file in tree
        if file["type"] == "blob" and file["path"].endswith(IMPORTANT_EXTENSIONS)
    ][:25]  # limit to 25 files

In [ ]:
def get_file_content(owner, repo, path):
    headers = {"Authorization": f"token {GITHUB_TOKEN}"}
    
    response = requests.get(
        f"https://api.github.com/repos/{owner}/{repo}/contents/{path}",
        headers=headers
    ).json()
    
    if "content" in response:
        import base64
        return base64.b64decode(response["content"]).decode("utf-8", errors="ignore")
    
    return ""

In [ ]:
client = OpenAI()

def generate_readme(repo_name, files_dict):
    
    context = ""
    for name, content in files_dict.items():
        context += f"\n\nFILE: {name}\n{content[:2000]}"
    
    prompt = f"""
You are a senior developer and technical writer.

Generate a professional, eye-catching README.md for a GitHub repository called "{repo_name}".

Use:
- Badges
- Clear feature list
- Installation instructions
- Usage examples
- Clean formatting
- Emojis tastefully
- Table of contents
- Contribution section
- License section

Here is the project context:

{context}
"""

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )

    return response.choices[0].message.content

In [ ]:
def create_readme(repo_url):
    try:
        owner, repo = parse_github_url(repo_url)
        tree = get_repo_tree(owner, repo)
        important_files = filter_files(tree)
        
        files_dict = {}
        for path in important_files:
            content = get_file_content(owner, repo, path)
            if content:
                files_dict[path] = content
        
        readme = generate_readme(repo, files_dict)
        return readme
    
    except Exception as e:
        return f"Error: {str(e)}"

with gr.Blocks() as app:
    gr.Markdown("# 🚀 AI README Generator")
    gr.Markdown("Paste a GitHub repo link and generate a professional README.")
    
    repo_input = gr.Textbox(label="GitHub Repo URL")
    output = gr.Markdown()
    
    generate_btn = gr.Button("Generate README")
    generate_btn.click(create_readme, repo_input, output)


In [ ]:
app.launch()